In [20]:
# %load 002_category_property.py
import gc
import numpy as np
import pandas as pd
from utils import BayesianSmoothing, load_pickle, dump_pickle, raw_data_path
from tqdm import tqdm
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
import time


start = time.time()
# path = '../data/'
# train_file = 'round1_ijcai_18_train_20180301.txt'
# test_file = 'round1_ijcai_18_test_a_20180301.txt'

# train = pd.read_table(path + train_file, encoding='utf8', delim_whitespace=True)
# test = pd.read_table(path + test_file, encoding='utf8', delim_whitespace=True)
# train.drop_duplicates('instance_id', inplace=True)
# test.drop_duplicates('instance_id', inplace=True)

# len_train = train.shape[0]
# df = train

train = pd.read_pickle('../data_valid/train.pkl')
test = pd.read_pickle('../data_valid/test.pkl')
df = pd.concat([train, test], axis=0)


print('done')

corpus = df.item_property_list.values.astype('U').tolist()
vectorizer = CountVectorizer()
vectorizer.fit(corpus)
countvector = vectorizer.transform(df.item_property_list)

pror_count = np.array(countvector.sum(axis=0))[0]
count = pd.Series(data=pror_count, index=np.arange(0, len(pror_count)))
selected_index = list(count.sort_values(ascending=False)[:10].index)
column_name = np.array(vectorizer.get_feature_names())[selected_index]
column_value = countvector[:, selected_index]

final_feat = pd.DataFrame(column_value.toarray(), columns=column_name)
final_feat['instance_id'] = df['instance_id']
final_feat['day'] = df['day']


train_feat = final_feat[(final_feat['day'] >= 18) & (final_feat['day'] <= 23)].drop('day', axis=1)
test_feat = final_feat[final_feat['day'] == 24].drop('day', axis=1)
print('the shape of train {}'.format(train_feat.shape))
print('the shape of test {}'.format(test_feat.shape))
dump_pickle(train_feat, path='../data_valid/train_feature/002_property_count.pkl')
dump_pickle(test_feat, path='../data_valid/test_feature/002_property_count.pkl')

end = time.time()
print('time elapsed {}'.format(end-start))

done
the shape of train (420676, 11)
the shape of test (57411, 11)
time elapsed 46.04399609565735


In [19]:
test_feat.head()

,2636395404473730413,5131280576272319091,9148482949976129397,6491818071284064879,1243768796732904674,4678095570925618478,2033679869864207699,7344985833148694227,2865114645280905956,1761412768692736998,instance_id
420676,1,0,0,0,0,0,0,0,0,0,2137051386648397465
420677,1,0,0,0,0,0,0,0,0,0,8636603765413923862
420678,1,1,0,1,1,0,0,1,1,0,345383580586199930
420679,1,1,0,1,1,0,0,1,1,0,8731890116021855314
420680,1,1,0,1,1,0,0,1,1,0,4313209774350468112
